# **Пример `.model_validate(data)` и `.model_dump()`**

In [183]:
from pydantic import BaseModel

class UserSchema(BaseModel):
    id: int = None
    name: str = None
    email: str = None
    test_field: str = None # Задали тестовое поле которого нет в словаре data

class MainUserSchema(BaseModel):
    items: list[UserSchema]

# Валидация из словаря
data = {"items": [{"id": 1, "name": "John Doe", "email": "john@example.com"},
                  {"id": 2, "name": "Ivan", "email": "ivan@example.com"}]}

user = MainUserSchema.model_validate(data)
user

MainUserSchema(items=[UserSchema(id=1, name='John Doe', email='john@example.com', test_field=None), UserSchema(id=2, name='Ivan', email='ivan@example.com', test_field=None)])

In [184]:
from pydantic import BaseModel

class UserSchema(BaseModel):
    id: int = None
    name: str = None
    email: str = None
    test_field: str = None # Задали тестовое поле которого нет в словаре data

class MainUserSchema(BaseModel):
    items: list[UserSchema]

# Валидация из словаря
data = {"items": [{"id": 1, "name": "John Doe", "email": "john@example.com"},
                  {"id": 2, "name": "Ivan", "email": "ivan@example.com"}]}
user = MainUserSchema.model_validate(data).model_dump(by_alias=True)
user

{'items': [{'id': 1,
   'name': 'John Doe',
   'email': 'john@example.com',
   'test_field': None},
  {'id': 2, 'name': 'Ivan', 'email': 'ivan@example.com', 'test_field': None}]}

In [185]:
'''
Аргумент by_alias=True в методе .model_dump() (или .dict() в Pydantic v1)
используется для управления именами ключей в итоговом словаре — выводить ли их как:
- Имена полей модели (по умолчанию)
или
- Заданные псевдонимы (alias)
'''

from pydantic import BaseModel, Field

class User(BaseModel):
    id: int = Field(alias="user_id")
    name: str

data = {
        "user_id": 100,
        "name": "Alice"
      }

user = User.model_validate(data)
print(user)

print(user.model_dump())
# {'id': 100, 'name': 'Alice'}

print(user.model_dump(by_alias=True))
# {'user_id': 100, 'name': 'Alice'}

id=100 name='Alice'
{'id': 100, 'name': 'Alice'}
{'user_id': 100, 'name': 'Alice'}


# **Пример с использованием API HH**

In [186]:
from pydantic import BaseModel   #, Field
from typing import Optional
import requests

class EmployerItemSchema(BaseModel):
    id: str
    name: str
    url: str = None
    alternate_url: str = None
    vacancies_url: str = None
    open_vacancies: int = None
    test_atr: str = 'Поле отсутствует' # Тестовое поле, если его нет в ответе от API, то выводит 'Поле отсутствует'

class EmployersListSchema(BaseModel):
    items: list[EmployerItemSchema]
    #found: int
    #pages: int
    #page: int

In [187]:
def get_employers_1():
    api_url = "https://api.hh.ru"
    url = f"{api_url}/employers"
    params = {
        'text': 'Дикси',
        'type': 'company',
        'area': 1,
        'only_with_vacancies': True,
    }

    response = requests.get(url, params=params)
    return response

response = get_employers_1()
# type(response.json()) -> dict
data = response.json()
data

{'items': [{'id': '2746',
   'name': 'ДИКСИ, группа компаний',
   'url': 'https://api.hh.ru/employers/2746',
   'alternate_url': 'https://hh.ru/employer/2746',
   'logo_urls': {'original': 'https://img.hhcdn.ru/employer-logo-original/1393191.png',
    '240': 'https://img.hhcdn.ru/employer-logo/7192548.png',
    '90': 'https://img.hhcdn.ru/employer-logo/7192547.png'},
   'vacancies_url': 'https://api.hh.ru/vacancies?employer_id=2746',
   'open_vacancies': 2898}],
 'found': 1,
 'pages': 1,
 'page': 0,
 'per_page': 20}

In [188]:
json_resume_list = []
if response.status_code == 200:
  try:
    # Валидация ответа от API
    validated_data = EmployersListSchema.model_validate(data).model_dump(by_alias=True)
    json_resume_list.append(validated_data)
    print(validated_data)

  except Exception as e:
    print("Ошибка валидации данных:", e)

else:
  print(f"Ошибка: {response.status_code}")

print('\nСписок json_resume_list:')
json_resume_list

{'items': [{'id': '2746', 'name': 'ДИКСИ, группа компаний', 'url': 'https://api.hh.ru/employers/2746', 'alternate_url': 'https://hh.ru/employer/2746', 'vacancies_url': 'https://api.hh.ru/vacancies?employer_id=2746', 'open_vacancies': 2898, 'test_atr': 'Поле отсутствует'}]}

Список json_resume_list:


[{'items': [{'id': '2746',
    'name': 'ДИКСИ, группа компаний',
    'url': 'https://api.hh.ru/employers/2746',
    'alternate_url': 'https://hh.ru/employer/2746',
    'vacancies_url': 'https://api.hh.ru/vacancies?employer_id=2746',
    'open_vacancies': 2898,
    'test_atr': 'Поле отсутствует'}]}]

# **class Config**

В Pydantic внутри модели можно определить вложенный класс `Config`, чтобы настроить поведение модели — например, как она валидирует данные, как сериализуется и т.д.

```
from pydantic import BaseModel

class User(BaseModel):
    name: str
    age: int

    class Config:
        title = "User Model"  # название модели (для документации)
        anystr_strip_whitespace = True  # обрезать пробелы у строк при валидации
        validate_assignment = True  # проверять данные при присваивании полям после создания объекта
        extra = "forbid"  # запретить лишние поля при инициализации (если передать что-то лишнее — будет ошибка)
```

In [191]:
from pydantic import BaseModel

class Resume(BaseModel):
    name: str
    email: str

    class Config:
        extra = "allow"

data = {
    "name": "Alice",
    "email": "alice@example.com",
    "age": 30,  # лишнее поле
    "location": "Paris"  # ещё одно лишнее поле
}

resume = Resume.model_validate(data)

print(resume.model_dump())  # {'name': 'Alice', 'email': 'alice@example.com', 'age': 30, 'location': 'Paris'}
print(resume.model_extra)   # {'age': 30, 'location': 'Paris'}


#extra	- Что делать с лишними полями: allow (разрешить), ignore (игнорировать), forbid (ошибка)

{'name': 'Alice', 'email': 'alice@example.com', 'age': 30, 'location': 'Paris'}
{'age': 30, 'location': 'Paris'}


# **Field**

Field — это способ настроить поведение поля в Pydantic.

Например:

Сделать другое имя в JSON

Ограничить длину строки

Написать описание

`...` — означает: поле обязательное

`min_length=2` — минимум 2 буквы

`max_length=10` — максимум 10 букв

`description="..."` — просто текст для документации

In [192]:
from pydantic import BaseModel, Field

class User(BaseModel):
    name: str = Field(..., min_length=2, max_length=10, description="Имя пользователя")

In [182]:
User(name="Al")

User(name='Al')

In [169]:
User(name="A") # Ошибка — слишком коротко

ValidationError: 1 validation error for User
name
  String should have at least 2 characters [type=string_too_short, input_value='A', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/string_too_short

In [171]:
User(name="Alexanderrrr") # Ошибка — слишком длинное имя

ValidationError: 1 validation error for User
name
  String should have at most 10 characters [type=string_too_long, input_value='Alexanderrrr', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/string_too_long

In [172]:
User() # Ошибка — не передано имя

ValidationError: 1 validation error for User
name
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

# **Optional**

Optional — это специальный тип из модуля typing, который указывает, что значение может быть либо указанного типа, либо None. Это важный инструмент для

Ключевые моменты:

`Optional[str]` означает: поле может быть `str` или `None`.

`= None` означает: если не передать поле, оно будет `None` по умолчанию.

In [193]:
from pydantic import BaseModel

class User(BaseModel):
    name: str  # обязательно

# Пример 1 — передаём name
user1 = User(name="Иван")
print(user1)

# Пример 2 — не передаём name
user2 = User()  # ❌ выдаст ошибку

name='Иван'


ValidationError: 1 validation error for User
name
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

In [194]:
from pydantic import BaseModel

class User(BaseModel):
    name: str = None  # обязательно

# Пример 1 — передаём name
user1 = User(name="Иван")
print(user1)

# Пример 2 — не передаём name
user2 = User()  # выдаст None
print(user2)

name='Иван'
name=None


In [195]:
from typing import Optional
from pydantic import BaseModel

class User(BaseModel):
    name: Optional[str]

user1 = User(name="Иван")
print(user1)

user2 = User()
print(user2)

name='Иван'


ValidationError: 1 validation error for User
name
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

In [196]:
from typing import Optional
from pydantic import BaseModel

class User(BaseModel):
    name: Optional[str] = None

user1 = User(name="Иван")
print(user1)

user2 = User()
print(user2)

name='Иван'
name=None
